In [ ]:
import requests
#import jieba
#from hanziconv.hanziconv import HanziConv
import sklearn_crfsuite
from sklearn_crfsuite import scorers, metrics
#from ckip import CkipSegmenter

###########################################################
def compare(actual_toks, pred_toks):
	i = 0
	j = 0
	p = 0
	q = 0
	tp = 0
	fp = 0

	##actual_toks, pred_toks is for one sentence
    
	while i < len(actual_toks) and j < len(pred_toks):
		if p == q:
			##compare by words, not characters
			if actual_toks[i] == pred_toks[j]:
				tp += 1
			else:
				fp += 1

			##p, q means pth or qth character in the sentence if combined
			p += len(actual_toks[i])
			q += len(pred_toks[j])

			##i,j means ith or jth word
			i += 1
			j += 1
		elif p < q:
			p += len(actual_toks[i])
			i += 1
		else:
			fp += 1
			q += len(pred_toks[j])
			j += 1

	#len(actual_toks)  vs how many words in a sentence
	return tp, fp, len(actual_toks)


    
def score(actual_sents, pred_sents):
	tp = 0
	fp = 0
	total = 0

	#actual_toks, pred_toks is for one sentence
	for actual_toks, pred_toks in zip(actual_sents, pred_sents):
		tp_, fp_, total_ = compare(actual_toks, pred_toks)
		tp += tp_
		fp += fp_
		total += total_
    
	recall = float(tp) / total
	precision = float(tp) / (tp + fp)
	f1 = 2.0 * recall * precision / (recall + precision)
	
	return recall, precision, f1

#############################################################

#use the algorithm of simplified chinese and use its sequence to restore back to traditional chinese
#the only function that can be omitted if you do not use simplified chinese

def restore(text, toks):
	results = []
	offset = 0
	for tok in toks:
		results.append(text[offset:offset + len(tok)])
		offset += len(tok)

	return results

#################################################
#x is list which has characters in the sentence

def extract_sent_features(x):
	sent_features = []
	for i in range(len(x)):
		sent_features.append(extract_char_features(x, i))
	
	return sent_features

##!!
def extract_char_features(sent, position):
	char_features = {}

	##{char_at_-3:zhong, char_at_-2:wen,.....,char_at_3:nan}
	##char_at is a relative position
	##return the former and latter character dict for a character

	for i in range(-3, 4):
		if len(sent) > position + i >= 0:
			char_features['char_at_%d' % i] = sent[position + i]
			
			###
			#result = segmenter.seg(sent[position + i])
			#char_features['pos_char_at_%d' % i]=result.pos[0]
			###

			###
			j=i+1
			if len(sent) > position + j >= 0:
                char_features['bigram_at_%d' % i] = "bigram_%s_%s" % (sent[position + i], sent[position + j])
            ###    
			#	check1=sent[position + i]+sent[position + j]
			#	try:
			#		result1 = segmenter.seg(check1)
			#		if len(result1.pos)==1:
			#			char_features['bigram_at_%d' % i] = "bigram_%s_%s" % (sent[position + i], sent[position + j])
			#	except:
			#		print("bigram doesn't exist in ckip")
			###

			###
			#	k=i+2
			#	if len(sent) > position + k >= 0:
			#		check2=sent[position + i]+sent[position + j]+sent[position + k]
			#		try:
			#			result2 = segmenter.seg(check2)
			#			if len(result2.pos)==1:
			#				char_features['trigram_at_%d' % i] = "trigram_%s_%s_%s" % (sent[position + i], sent[position + j],sent[position + k])
			#		except:
			#			print("trigram doesn't exist in ckip")
			###



	
	return char_features
#########################################################

##words is list of words it cut based on spaces in the original data

def words_to_tags(words):
	##tag the sequence of each character
	tags = []

	##each word may contain many chracters
	##one word ->S
	##more than one word->LMMMR
	for word in words:
		if len(word) == 1:
			tags.append('S')
		else:
			for i in range(len(word)):
				if i == 0:
					tags.append('L')
				elif i == len(word) - 1:
					tags.append('R')
				else:
					tags.append('M')
	return tags

#####################################################################

def segment(sent):
	##sent is a sentence which characters are connected
	##we list them into ["zhong","wen","how","nan"]
	##extract_sent_feature:[{},{},{}]
	##predict_single:tags=['L','M','M','R']

	tags = crf_tagger.predict_single(extract_sent_features(list(sent)))
	tokens = []
	tok = ""

	for ch, tag in zip(list(sent), tags):
		##if it encounters a 'L' or 'S', it adds the previous accumulated characters(a word) before it adds the new character 
		if tag in ['S', 'L'] and tok != "":
			tokens.append(tok)
			tok = ""
		##if it is 'M' or 'R', you accumulate the character
		tok += ch

	##tokens is for returning words (put some characters together) based on your label
	##if there is something left, we add it
	if tok:
		tokens.append(tok)
	
	return tokens

###################################################################
#def clean(string): 
#	string = string.replace("，","")
#	string = string.replace("。","")
#	string = string.replace("、","")
#	string = string.replace("!","")
#	string = string.replace("？","")
	
#	string = string.replace("~"," ") 
#	string = string.replace("：","")

#	string = string.replace("[","")
#	string = string.replace("]","")

#	string = string.replace("「","")
#	string = string.replace("」","")

#	string = string.replace("（","")
#	string = string.replace("）","")
#	string = string.replace("／","")

#	string = string.replace("；","")

#	string=string.strip()
#	string=string.split()

#	return string


####################################################################################
#input
remote_url = "https://raw.githubusercontent.com/hhhuang/nlp2019fall/master/word_segmentation/"
r=requests.get(remote_url + "data/as_training.utf8", allow_redirects=True)
open('as_training.utf8', 'wb').write(r.content)

r=requests.get(remote_url + "data/as_testing_gold.utf8", allow_redirects=True)
open('as_testing_gold.utf8', 'wb').write(r.content)

#segmenter = CkipSegmenter()
#################################################
##in raw data , the split is based on spaces
##it has spaces originally

raw_train = []
raw_test = []  ##contain many lists, each list is ['chinese word', 'chinese word','comma'...]
with open("as_training.utf8", encoding="utf8") as fin:
	for line in fin:
		raw_train.append(line.strip().split("　"))
		#clean_train=clean(line)
		#raw_train.append(clean_train)   # It is a full white space


with open("as_testing_gold.utf8", encoding="utf8") as fin:
	for line in fin:
		raw_test.append(line.strip().split("　"))
		#clean_test=clean(line)
		#raw_test.append(clean_test)   # It is a full white space

##############################################################################
#print("Number of sentences in the training data: %d" % len(raw_train))
#print("Number of sentences in the test data: %d" % len(raw_test))

##our goal is to cut the word correctly for a sentence
##simplified chinese is better than traditional ones
#print(list(jieba.cut("".join(raw_test[0]))))
#print(list(jieba.cut(HanziConv.toSimplified("".join(raw_test[0])))))

##use simplified chinese algo. results and restore back to traditional chinese
#text = "".join(raw_test[0])
#print(restore(text, list(jieba.cut(HanziConv.toSimplified(text)))))
##########################################################

train_X = []
train_Y = []

#test_X = []
#test_Y = []

###
##print(sent)    #['zhongwen','hownan']   ##based on spaces it cut in the original data
##print("".join(sent))  #zhongwenhownan
##print(list("".join(sent)))  #['zhong','wen','how','nan']
##text_x=[['zhong','wen','how','nan'],['how',bann]]
###


##raw_train contains many lists, each list of words has already been split originally based on spaces
for sent in raw_train:
	train_X.append(list("".join(sent)))  # Make the unsegmented sentence as a sequence of characters
	train_Y.append(words_to_tags(sent))


##didn't use test_X, test_Y after
#for sent in raw_test:
#	test_X.append(list("".join(sent)))  # Make the unsegmented sentence
#	test_Y.append(words_to_tags(sent))

##
##test_x->test_y is one character to {S,L,M,R}
##
#print(raw_test[0])    
#print(test_X[0])
#print(test_Y[0])
##

############################################################################

feature_X = []
for x in train_X:
	feature_X.append(extract_sent_features(x))


##!!
crf_tagger = sklearn_crfsuite.CRF(algorithm='lbfgs', min_freq=20, max_iterations=300, verbose=True)
##feature_X: [[{},{},{}...],[{},{}..]] each dict is the former 3 and latter four characters for one chracter
##train_Y: [['L','R','L','R'],['L','M','M','R']]  each list is one sentence, each label is for one character
crf_tagger.fit(feature_X, train_Y)

#print(segment("法國總統馬克宏已到現場勘災，初步傳出火警可能與目前聖母院的維修工程有關。"))
##############################
##method1

pred = []

##how it cut the words in the original data  #it is a list
actual = []

##compare model prediction with your original answer
for sent in raw_test:
	pred.append(segment("".join(sent)))
	actual.append(sent)


print(actual[0])
print(pred[0])
print(score(actual, pred))


######################################################################
#method2
#if you use jiba api

#pred = []
#actual = []

##for you to see  #can work without fout 
#fout = open("jieba.out", "w", encoding="utf-8")

#for sent in raw_test:
#	text = "".join(sent)
#	r = list(jieba.cut(HanziConv.toSimplified(text)))
#	r = restore(text, r)
#	fout.write(" ".join(r) + "\n")
#	pred.append(r)
#	actual.append(sent)

#print(actual[0])
#print(pred[0])
#print(score(actual, pred))

###########################
